# Linking Filter Validation

In [ ]:
import pandas as pd
import numpy as np
from sorcha.modules.PPLinkingFilter import PPLinkingFilter

This function aims to mimic the effects of the Solar System Processing pipeline in linking objects. More information can be found [here](http://lsst-sssc.github.io/software.html). If we use the SSP defaults, for an object to be linked, it must have:
* At least **2** observations in a night to constitute a valid tracklet.
* These observations must have an angular separation of at least **0.5 arcseconds** in order to be recognised as separate.
* However, subsequent observations in a tracklet must occur within 90 minutes or **0.0625 days**.
* At least **3** tracklets must be observed to form a valid track.
* These tracklets must be observed in less than **15** days.

We also expect **95%** of objects to be linked. For now, we will set this parameter to 100% in order to test the others.

These six parameters can be changed in the config file and are found in the [LINKINGFILTER] section.

In [ ]:
min_observations = 2
min_angular_separation = 0.5
max_time_separation = 0.0625
min_tracklets = 3
min_tracklet_window = 15
detection_efficiency = 1

Let's create an object that should definitely be linked according to these parameters.

In [ ]:
obj_id = ["pretend_object"] * 6
field_id = np.arange(1, 7)
times = [60000.03, 60000.06, 60005.03, 60005.06, 60008.03, 60008.06]
ra = [142, 142.1, 143, 143.1, 144, 144.1]
dec = [8, 8.1, 9, 9.1, 10, 10.1]

In [ ]:
observations = pd.DataFrame(
    {
        "ObjID": obj_id,
        "FieldID": field_id,
        "FieldMJD_TAI": times,
        "AstRA(deg)": ra,
        "AstDec(deg)": dec
    }
)

In [ ]:
observations

Now let's run the linking filter. As this object should be linked, we should receive the same dataframe back.

In [ ]:
linked_observations = PPLinkingFilter(observations, detection_efficiency, min_observations, min_tracklets, min_tracklet_window, min_angular_separation, max_time_separation)

In [ ]:
linked_observations

Success! The object was successfully linked. Now let's play with this dataframe a little. First, let's remove the first observation, so that we only have two complete tracklets.

In [ ]:
observations_two_tracklets = observations.iloc[1:].copy()

In [ ]:
observations_two_tracklets

In [ ]:
unlinked_observations = PPLinkingFilter(observations_two_tracklets, detection_efficiency, min_observations, min_tracklets, min_tracklet_window, min_angular_separation, max_time_separation)

In [ ]:
unlinked_observations

As expected, we no longer link the object. Now let's try putting the last two observations outside of the 15-day window.

In [ ]:
observations_large_window = observations.copy()
observations_large_window['FieldMJD_TAI'] = [60000.03, 60000.06, 60005.03, 60005.06, 60016.03, 60016.06]

In [ ]:
unlinked_observations = PPLinkingFilter(observations_large_window, detection_efficiency, min_observations, min_tracklets, min_tracklet_window, min_angular_separation, max_time_separation)

In [ ]:
unlinked_observations

Once again, we no longer link the object. What if we move the first two objects much closer to each other so that they no longer form a valid tracklet?

In [ ]:
observations_small_sep = observations.copy()
observations_small_sep["AstRA(deg)"] = [142, 142.00001, 143, 143.1, 144, 144.1]
observations_small_sep["AstDec(deg)"] = [8, 8.00001, 9, 9.1, 10, 10.1]

In [ ]:
unlinked_observations = PPLinkingFilter(observations_small_sep, detection_efficiency, min_observations, min_tracklets, min_tracklet_window, min_angular_separation, max_time_separation)

In [ ]:
unlinked_observations

And the object is no longer linked. Finally, let's move the first two observations much further apart in time so that they once again no longer form a valid tracklet.

In [ ]:
observations_large_time = observations.copy()
observations_large_time["FieldMJD_TAI"] = [60000.03, 60000.10, 60005.03, 60005.06, 60008.03, 60008.06]

In [ ]:
unlinked_observations = PPLinkingFilter(observations_large_time, detection_efficiency, min_observations, min_tracklets, min_tracklet_window, min_angular_separation, max_time_separation)

In [ ]:
unlinked_observations

And as expected, we no longer link the object.

Finally, let's check that the detection efficiency works as expected. Let's set it to 0.95.

In [ ]:
detection_efficiency = 0.95

Now let's make a dataframe of the same linked object repeated 10000 times.

In [ ]:
objs = [["pretend_object_" + str(a)] * 6 for a in range(0, 10000)]
obj_id_long = [item for sublist in objs for item in sublist]
field_id_long = list(np.arange(1, 7)) * 10000
times_long = [60000.03, 60000.06, 60005.03, 60005.06, 60008.03, 60008.06] * 10000
ra_long = [142, 142.1, 143, 143.1, 144, 144.1] * 10000
dec_long = [8, 8.1, 9, 9.1, 10, 10.1] * 10000

In [ ]:
observations_long = pd.DataFrame(
    {
        "ObjID": obj_id_long,
        "FieldID": field_id_long,
        "FieldMJD_TAI": times_long,
        "AstRA(deg)": ra_long,
        "AstDec(deg)": dec_long
    }
)

In [ ]:
observations_long

If detection efficiency were perfect, all of these objects would be linked. However, it is not. We have set the detection efficency to 0.95, so we should expect to return roughly 95% of these objects from the linking filter. Let's find out.

In [ ]:
long_linked_observations = PPLinkingFilter(observations_long, detection_efficiency, min_observations, min_tracklets, min_tracklet_window, min_angular_separation, max_time_separation)

In [ ]:
long_linked_observations

In [ ]:
len(long_linked_observations["ObjID"].unique())/10000

This is close enough - the detection efficiency is stochastic, so some variation is to be expected.